T5 Flan
----
This notebook follows this guide: https://huggingface.co/docs/transformers/tasks/summarization

# Setup

In [1]:
! pip install transformers datasets evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 20.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.9/769.9 kB 21.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 21.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.0/39.0 MB 19.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, TFAutoModelForSeq2SeqLM, create_optimizer, AdamWeightDecay, pipeline
from transformers.keras_callbacks import KerasMetricCallback
import evaluate
import numpy as np

Defining the model to use.

In [3]:
checkpoint = 'google/flan-t5-small'

# Data

In [4]:
billsum = load_dataset('billsum', split='ca_test').train_test_split(test_size=0.2)

# HF data objects can be indexed EITHER by obs or key: the former returns a dict, the latter a list
print(billsum['train'][0])
print(billsum['train']['summary'][:5])

Generating ca_test split: 100%|██████████| 1237/1237 [00:00<00:00, 10245.06 examples/s]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 82013 of the Government Code is amended to read:\n82013.\n“Committee” means any person or combination of persons who directly or indirectly does any of the following:\n(a) Receives contributions totaling two thousand dollars ($2,000) or more in a calendar year.\n(b) Makes independent expenditures totaling one thousand dollars ($1,000) or more in a calendar year; or\n(c) Makes contributions totaling ten thousand dollars ($10,000) or more in a calendar year to or at the behest of candidates or committees.\nA person or combination of persons that becomes a committee shall retain its status as a committee until such time as that status is terminated pursuant to Section 84214.\nSEC. 2.\nSection 82036 of the Government Code is amended to read:\n82036.\n“Late contribution” means any of the following:\n(a) A contribution, including a loan, that totals in the aggregate one thousand dollars ($1,000) or 

In [5]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

c:\Users\marks\.virtualenvs\QualAbstracts-_C7lXhsk\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\marks\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [6]:
def preprocess_function(data):
    prefix = "summarize: "
    inputs = [prefix + text for text in data['text']]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    
    labels = tokenizer(text_target=data['summary'], max_length=128, truncation=True)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [7]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=checkpoint,
    return_tensors='tf'
)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map: 100%|██████████| 248/248 [00:01<00:00, 222.75 examples/s]


# Defining Model and Metrics

In [8]:
rouge = evaluate.load('rouge')
def compute_metrics(eval_pred, evalutor=rouge, tokenizer=tokenizer):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id) # not sure what this is doing
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = evalutor.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}


Note that HF models have built-in loss functions, so one does not need to be specified when compiling.

In [9]:
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)
model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)
model.compile(optimizer=optimizer)

train_set = model.prepare_tf_dataset(
    tokenized_billsum['train'],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator
)
test_set = model.prepare_tf_dataset(
    tokenized_billsum['test'],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator
)

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=test_set)

model.fit(
    x=train_set,
    validation_data=test_set,
    epochs=3,
    callbacks=[metric_callback]
)

c:\Users\marks\.virtualenvs\QualAbstracts-_C7lXhsk\lib\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at google/flan-t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to ge

Epoch 1/3
 1/61 [..............................] - ETA: 2:14:28 - loss: 2.9154

: 

: 